In [13]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import normalize, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import mode
from sklearn.metrics import accuracy_score

In [14]:
datos = pd.read_csv('mnist_data.csv')
datos

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
label = pd.read_csv('mnist_data_label.csv')
label

,class
0,5
1,0
2,4
3,1
4,9
...,...
595,9
596,0
597,6
598,6


In [16]:
class neurona(object):
    def __init__(self, f=0, c=0, dim=0):
        self.c = c
        self.f = f
        self.dim = dim
        
        # Inicializa los pesos aleatoriamente entre [-0.5, 0.5]
        self.w = normalize(0.5 - np.random.rand(dim).reshape(1,-1))
        
        # self.label_winner = []
            
    def predict(self, inputs):
        # Calcula la salida de una neurona ante una o más entradas. "inputs" puede ser un vector o una matriz 2D
        return inputs @ self.w.T
    
    def fit(self, input, alfa=1):
        # ajusta los pesos de una neurona (w) para aproximarlos a una entrada (input)
        self.w = normalize(self.w + (alfa*input))
            
    def neuron_labeling(self, inputs, target):
        # etiquetado por neuronas. Se le pasa la lista de entradas y la etiqueta (target) de cada una de esas muestras.
        # Devuelve la etiqueta de la ganadora
        Y = inputs @ self.w.T
        self.label = target[np.argmax(Y)]
        return self.label

In [17]:
class som():
    
    def __init__(self, filas=1, columnas=1, dim=1):
        self.lista = []
        self.filas = filas
        self.columnas = columnas
        self.dim = dim
        
        # Considera que un mapa rectangular es una lista de objetos "neurona", que viene localizado por sus atributos "fila" y "columna"
        for fila in range(self.filas):
            for columna in range(self.columnas):
                self.lista.append(neurona(f=fila, c=columna, dim=dim))
                
    def fit(self, inputs, max_epochs=1, init_radious=0, init_alfa=1):
        # método similar a otros algoritmo de ML. Recibe las entradas, el radio inicial, el factor de apendizaje inicial,
        # el máximo de épocas y devuelve los pesos ajustados
        self.radious = init_radious
        self.alfa = init_alfa
        t = 0
        P = inputs.shape[0]
        for epoch in range(max_epochs):
            for x in inputs:
                self.alfa = init_alfa/(1.0 + float(t/P))
                i_gana, y_gana = -1, float('-inf')
                for i in range(self.filas*self.columnas):
                    y_predict = self.lista[i].predict(x.reshape(1,-1))
                    if y_predict > y_gana:
                        y_gana = y_predict
                        i_gana = i
                f_gana = int(i_gana / self.columnas)
                c_gana = i_gana % self.columnas
                
                # Conjunto de vecinas para un radious
                for f in range(f_gana - self.radious, f_gana + self.radious+1):
                    if f < 0:
                        row = self.filas + f
                    else:
                        if f > self.filas-1:
                            row = f % self.filas
                        else:
                            row = f

                    for c in range(c_gana - self.radious, c_gana + self.radious+1):
                        if c < 0:
                            column = self.columnas + c 
                        else:
                            if c > self.columnas-1:
                                column = c % self.columnas
                            else:
                                column = c
                        self.lista[(row*self.columnas) + column].fit(x.reshape(1,-1), alfa=self.alfa)
                t += 1
                if (t%1000) == 0:
                    print(t, self.radious, "  ", end='')
            if self.radious > 0:
                self.radious -= 1
                            
    def neuron_labeling(self, inputs, target):
        # recorre la lista de neuronas y va llamanado a su metodo de etiquetado para cada neurona
        self.labels = []
        
        for i in range(self.filas*self.columnas):
            # print(X.shape, self.target.shape)
            self.labels.append(self.lista[i].neuron_labeling(inputs, target))
            # print(self.lista[i].labeling(X, target=y_deseada, etiquetado='neurona'))
 
    def predict(self, inputs):
        # recorre la lista de neuronas y calcula la salida de un conjunto de muestras
        # util para usar la salida del som como entrada a otrso sistemas
        output_list = []
        for x in inputs:
            for i in range(self.filas*self.columnas):
                output_list.append(self.lista[i].predict(x.reshape(1,-1)))
        return np.array(output_list).reshape(inputs.shape[0], -1)    
    
    def label_predict(self, inputs):
        # clasificación de muestras con el etiquetado de cada neurona hecho previamente
        label_list = []
        for x in inputs:
            label_list.append(self.labels[np.argmax(self.predict(x.reshape(1,-1)))])
        return np.array(label_list).reshape(inputs.shape[0], -1)

In [18]:
# Normalizacion datos
scalar = StandardScaler()
datos_scd = scalar.fit_transform(datos)
datos_scd

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
train_data, test_data, train_label, test_label = train_test_split(datos_scd, label, test_size=0.33)

In [20]:
train_data.shape

(402, 784)

In [21]:
test_data.shape

(198, 784)

In [22]:
# Creacion de la red neuronal SOM
mapa = som(15,9, train_data.shape[1])

# Entrenamos la red neuronal
mapa.fit(train_data, max_epochs=20, init_radious=9, init_alfa=10)

1000 7   2000 5   3000 2   4000 0   5000 0   6000 0   7000 0   8000 0   

In [23]:
mapa.neuron_labeling(train_data, train_label.values)

In [24]:
neuron_labels = mapa.label_predict(test_data)
neuron_labels.shape

(198, 1)

In [25]:
accuracy_score(test_label, neuron_labels)

0.7575757575757576

In [26]:
neuron_labels_train = mapa.label_predict(train_data)
neuron_labels_train.shape

(402, 1)

In [27]:
train = mapa.predict(train_data)
train

array([[-3.54691106,  3.22977999,  2.12704152, ...,  0.16801853,
         1.7955824 , -2.13300856],
       [-0.50908424, -1.62787544, -0.33405356, ...,  0.76368931,
        -1.55760863, -1.09064992],
       [ 2.47758238, -0.1653203 , -2.36044903, ...,  0.52376146,
        -1.2012504 ,  1.62740307],
       ...,
       [ 9.10262222, -0.61579027, 10.52283699, ..., -2.0452204 ,
        -1.6627318 , -4.23009026],
       [ 0.40445744, -2.13448259, -1.78802246, ...,  0.10878085,
        -0.11137002,  3.01076018],
       [ 2.20539033,  0.86737905, -2.34669948, ...,  0.12105293,
        -0.91738819,  2.6373069 ]])

In [28]:
test = mapa.predict(test_data)
test

array([[ 0.15101479, -0.7500538 , -1.09064676, ..., -1.29020001,
         1.44942873, -1.42014029],
       [ 0.88617784, -2.50313742, -3.03373723, ..., -0.70065123,
         4.61561328,  1.42136777],
       [ 0.03992838, -1.08432602, -1.70341215, ...,  0.21127945,
        -0.48244536,  1.13952869],
       ...,
       [ 0.09738522, -1.02950055, -1.41028267, ...,  0.20244095,
        -0.63628544,  0.84098915],
       [ 0.36075448, -1.36341332, -0.73143561, ...,  0.55283373,
        -1.01247431,  1.4539431 ],
       [ 0.14943117, -1.94263085, -1.5710607 , ...,  0.17664078,
        -0.79467038,  1.20339768]])

In [29]:
from sklearn.neural_network import MLPClassifier

In [30]:
clf = MLPClassifier(hidden_layer_sizes=(60,), activation='relu', solver='adam', random_state=1)

In [31]:
train.shape, neuron_labels_train.shape

((402, 135), (402, 1))

In [32]:
clf.fit(train, neuron_labels_train)

C:\Users\Jhon\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MLPClassifier(hidden_layer_sizes=(60,), random_state=1)

In [33]:
y_predict = clf.predict(test)

In [34]:
y_predict

array([4, 5, 1, 6, 3, 4, 2, 3, 2, 5, 4, 4, 9, 4, 8, 2, 5, 4, 0, 4, 6, 3,
       1, 0, 4, 1, 7, 6, 2, 9, 4, 0, 7, 3, 3, 9, 6, 1, 1, 5, 3, 7, 4, 3,
       5, 0, 9, 7, 2, 2, 5, 1, 6, 5, 4, 1, 1, 4, 1, 6, 1, 1, 1, 8, 9, 1,
       1, 5, 6, 9, 9, 0, 5, 4, 3, 6, 3, 8, 0, 9, 1, 9, 7, 7, 7, 2, 0, 1,
       2, 1, 8, 4, 3, 1, 1, 9, 0, 1, 4, 2, 2, 6, 8, 6, 3, 6, 7, 3, 2, 3,
       4, 2, 0, 6, 9, 9, 5, 2, 7, 2, 6, 1, 2, 4, 0, 2, 1, 7, 9, 5, 8, 1,
       7, 0, 0, 4, 9, 5, 1, 6, 8, 6, 6, 2, 3, 8, 9, 7, 6, 4, 1, 8, 4, 5,
       1, 8, 1, 0, 6, 0, 4, 7, 1, 4, 6, 9, 5, 8, 0, 0, 6, 5, 6, 7, 3, 9,
       0, 0, 0, 8, 7, 3, 1, 2, 7, 8, 4, 7, 9, 4, 7, 0, 1, 2, 8, 4, 1, 1],
      dtype=int64)

In [35]:
accuracy_score(test_label, y_predict)

0.7878787878787878